In [1]:
# This link might be useful, unclear yet: https://medium.com/randomai/ensemble-and-store-models-in-keras-2-x-b881a6d7693f
import numpy as np
import keras
from keras import backend
from keras.models import load_model
import tensorflow as tf
from utils import *

import cleverhans.attacks as Attacks
from cleverhans.attacks import FastGradientMethod
from cleverhans.attacks import BasicIterativeMethod
from cleverhans.utils_keras import KerasModelWrapper

Using TensorFlow backend.
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Set up

In [2]:
ATTACK_NAME = 'fgsm' # sys.argv[1]
DATASET_NAME = 'mnist' # sys.argv[2]

In [3]:
# l-inf attacks
if ATTACK_NAME == 'fgsm':
    AttackModel = Attacks.FastGradientMethod
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0
    }
elif ATTACK_NAME == 'pgd':
    AttackModel = Attacks.ProjectedGradientDescent
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0
    }
elif ATTACK_NAME == 'bim':
    # subclass of pgd (rand_init == 0)
    AttackModel = Attacks.BasicIterativeMethod
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0
    }
# l-2 
elif ATTACK_NAME == 'inf-pgd':
    AttackModel = Attacks.ProjectedGradientDescent
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0,
    'ord': np.inf
    }
elif ATTACK_NAME == 'cw':
    # subclass of pgd (rand_init == 0)
    AttackModel = Attacks.CarliniWagnerL2
    attack_params = {
    'eps': 0.5,
    'clip_min': 0.0,
    'clip_max': 1.0
    }




In [4]:
num_classes = 10
(x_train, y_train, x_test, y_test) = get_dataset(DATASET_NAME)

x_train shape (60000, 28, 28, 1)
y_train shape (60000,)
x_test shape (10000, 28, 28, 1)
y_test shape (10000,)


In [5]:
backend.set_learning_phase(False)
sess =  backend.get_session()

# Define input TF placeholder
if DATASET_NAME == 'mnist':
    x = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
    y = tf.placeholder(tf.float32, shape=(None, 10))
elif DATASET_NAME == 'cifar10':
    x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3))
    y = tf.placeholder(tf.float32, shape=(None, 10))

In [18]:
def test_keras_model(model, SEED, images, labels):
    x_shuffle = permute_pixels(images, SEED)
    pred = np.argmax(model.predict(x_shuffle), axis = 1)
    acc =  np.mean(np.equal(pred, labels))
    return (pred, acc)

## Generate Adversarial Examples

In [19]:
KNOWN_SEED = 87
# we're going to give this model trained with KNOWN_SEED to the adversary

known_model = load_model('models/'+DATASET_NAME+'_trained_keras_model'+'.hdf5', custom_objects={'tf':tf}) 
pred, acc = test_keras_model(known_model, KNOWN_SEED, x_test, y_test)
print("The normal test accuracy is: {}".format(acc))

The normal test accuracy is: 0.9526


In [20]:
# generate adversariale examples (x_adv) using the known model
# http://everettsprojects.com/2018/01/30/mnist-adversarial-examples.html
# https://cleverhans.readthedocs.io/en/latest/source/attacks.html#generate_np

wrap = KerasModelWrapper(keras_model)
attack_model = AttackModel(wrap, sess=sess)
x_adv = attack_model.generate_np(x_test, **attack_params)

# test x_adv against the single model
pred, acc = test_keras_model(known_model, KNOWN_SEED, x_adv, y_test)
print("The adversarial test accuracy is: {}".format(acc))

The adversarial test accuracy is: 0.4881


## Attack Ensemble PPD models

In [23]:
# test x_adv against the ensemble model

num_models = 10  # debugging with smaller number. change this to 10/50 later.
num_samples = x_adv.shape[0]
adv_acc = []
normal_acc = []
# We're assuming majority voting?
# we're going to store votes from each model here
adv_pred = np.zeros((num_samples, num_classes)) 
normal_pred = np.zeros((num_samples, num_classes)) 

for SECRET_SEED in range(num_models):
    keras_model = load_model('models/'+DATASET_NAME+'_trained_keras_model_'+str(SECRET_SEED)+'.hdf5', custom_objects={'tf':tf})

    pred, acc = test_keras_model(keras_model, SECRET_SEED, x_test, y_test)
    normal_pred[range(num_samples), pred] += 1  # +1 vote 
    print ('SECRET_SEED:', SECRET_SEED, 'Individual model normal accuracy:', acc)
    normal_acc.append(acc)
    
    pred, acc = test_keras_model(keras_model, SECRET_SEED, x_adv, y_test)
    adv_pred[range(num_samples), pred] += 1  # +1 vote 
    print ('SECRET_SEED:', SECRET_SEED, 'Individual model adversarial accuracy:', acc)   
    adv_acc.append(acc)  # accuracy per model, not reported in paper

    
# for each sample, find out the class with most votes
ensemble_pred = np.argmax(normal_pred, axis = 1)
acc =  np.mean(np.equal(ensemble_pred, y_test))
print ('Ensemble normal accuracy:', acc)

ensemble_pred = np.argmax(adv_pred, axis = 1)
acc =  np.mean(np.equal(ensemble_pred, y_test))
print ('Ensemble adversarial accuracy:', acc)

SECRET_SEED: 0 Individual model normal accuracy: 0.9518
SECRET_SEED: 0 Individual model adversarial accuracy: 0.5326
SECRET_SEED: 1 Individual model normal accuracy: 0.9521
SECRET_SEED: 1 Individual model adversarial accuracy: 0.5099
SECRET_SEED: 2 Individual model normal accuracy: 0.9555
SECRET_SEED: 2 Individual model adversarial accuracy: 0.52
SECRET_SEED: 3 Individual model normal accuracy: 0.9539
SECRET_SEED: 3 Individual model adversarial accuracy: 0.5573
SECRET_SEED: 4 Individual model normal accuracy: 0.9521
SECRET_SEED: 4 Individual model adversarial accuracy: 0.5254
SECRET_SEED: 5 Individual model normal accuracy: 0.9549
SECRET_SEED: 5 Individual model adversarial accuracy: 0.532
SECRET_SEED: 6 Individual model normal accuracy: 0.9542
SECRET_SEED: 6 Individual model adversarial accuracy: 0.5268
SECRET_SEED: 7 Individual model normal accuracy: 0.9547
SECRET_SEED: 7 Individual model adversarial accuracy: 0.5144
SECRET_SEED: 8 Individual model normal accuracy: 0.9556
SECRET_SEED